# Innocean USA Advertising Intelligence - ML Models

This notebook trains and registers three ML models for the Innocean Intelligence solution:

1. **Campaign Performance Predictor** - Predict campaign success likelihood
2. **Client Churn Predictor** - Identify clients at risk of leaving
3. **Budget Optimization Model** - Recommend optimal media channel allocation

## Prerequisites
- Execute SQL scripts 01-06 to set up database and populate data
- Add required packages: `snowflake-ml-python`, `scikit-learn`, `xgboost`, `matplotlib`

In [ ]:
# Core imports
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
import numpy as np

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Snowflake ML imports
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.preprocessing import OneHotEncoder

# Visualization
import matplotlib.pyplot as plt

# Get active session
session = get_active_session()
print(f"Connected to: {session.get_current_database()}.{session.get_current_schema()}")

In [ ]:
%%sql -r campaign_data
SELECT 
    c.CAMPAIGN_ID,
    c.CAMPAIGN_TYPE,
    c.CAMPAIGN_OBJECTIVE,
    c.TOTAL_BUDGET,
    c.MEDIA_BUDGET,
    c.PRODUCTION_BUDGET,
    c.TARGET_IMPRESSIONS,
    c.TARGET_ROAS,
    c.ACTUAL_IMPRESSIONS,
    c.ACTUAL_ROAS,
    cl.CLIENT_SEGMENT,
    cl.RELATIONSHIP_TYPE,
    cl.SATISFACTION_SCORE,
    cl.NPS_SCORE,
    cl.ANNUAL_BUDGET AS CLIENT_ANNUAL_BUDGET,
    CASE 
        WHEN c.ACTUAL_ROAS >= c.TARGET_ROAS THEN 1 
        ELSE 0 
    END AS SUCCESS_LABEL
FROM INNOCEAN_INTELLIGENCE.RAW.CAMPAIGNS c
JOIN INNOCEAN_INTELLIGENCE.RAW.CLIENTS cl ON c.CLIENT_ID = cl.CLIENT_ID
WHERE c.STATUS = 'COMPLETED'
AND c.ACTUAL_ROAS IS NOT NULL
AND c.TARGET_ROAS > 0
LIMIT 50000

In [ ]:
# Prepare features for campaign performance prediction
df_campaign = campaign_data.copy()
print(f"Campaign records: {len(df_campaign)}")

# Feature engineering
df_campaign['BUDGET_RATIO'] = df_campaign['MEDIA_BUDGET'] / df_campaign['TOTAL_BUDGET'].replace(0, 1)
df_campaign['PRODUCTION_RATIO'] = df_campaign['PRODUCTION_BUDGET'] / df_campaign['TOTAL_BUDGET'].replace(0, 1)

# Encode categorical variables
le_type = LabelEncoder()
le_objective = LabelEncoder()
le_segment = LabelEncoder()
le_relationship = LabelEncoder()

df_campaign['CAMPAIGN_TYPE_ENC'] = le_type.fit_transform(df_campaign['CAMPAIGN_TYPE'].fillna('UNKNOWN'))
df_campaign['CAMPAIGN_OBJECTIVE_ENC'] = le_objective.fit_transform(df_campaign['CAMPAIGN_OBJECTIVE'].fillna('UNKNOWN'))
df_campaign['CLIENT_SEGMENT_ENC'] = le_segment.fit_transform(df_campaign['CLIENT_SEGMENT'].fillna('UNKNOWN'))
df_campaign['RELATIONSHIP_TYPE_ENC'] = le_relationship.fit_transform(df_campaign['RELATIONSHIP_TYPE'].fillna('UNKNOWN'))

# Select features
features = [
    'TOTAL_BUDGET', 'MEDIA_BUDGET', 'BUDGET_RATIO', 'PRODUCTION_RATIO',
    'TARGET_IMPRESSIONS', 'TARGET_ROAS', 'SATISFACTION_SCORE', 'NPS_SCORE',
    'CLIENT_ANNUAL_BUDGET', 'CAMPAIGN_TYPE_ENC', 'CAMPAIGN_OBJECTIVE_ENC',
    'CLIENT_SEGMENT_ENC', 'RELATIONSHIP_TYPE_ENC'
]

X = df_campaign[features].fillna(0)
y = df_campaign['SUCCESS_LABEL']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
campaign_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
campaign_model.fit(X_train, y_train)

# Evaluate
y_pred = campaign_model.predict(X_test)
y_prob = campaign_model.predict_proba(X_test)[:, 1]

print(f"\nCampaign Performance Model Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.3f}")
print(f"\nFeature Importance:")
for feat, imp in sorted(zip(features, campaign_model.feature_importances_), key=lambda x: -x[1])[:5]:
    print(f"  {feat}: {imp:.3f}")

In [ ]:
%%sql -r client_data
SELECT 
    cl.CLIENT_ID,
    cl.CLIENT_SEGMENT,
    cl.RELATIONSHIP_TYPE,
    cl.ANNUAL_BUDGET,
    cl.SATISFACTION_SCORE,
    cl.NPS_SCORE,
    DATEDIFF(DAY, cl.CONTRACT_START_DATE, CURRENT_DATE()) AS TENURE_DAYS,
    DATEDIFF(DAY, CURRENT_DATE(), cl.CONTRACT_END_DATE) AS DAYS_TO_CONTRACT_END,
    COUNT(DISTINCT c.CAMPAIGN_ID) AS CAMPAIGN_COUNT,
    SUM(c.TOTAL_BUDGET) AS TOTAL_CAMPAIGN_SPEND,
    AVG(c.ACTUAL_ROAS) AS AVG_CAMPAIGN_ROAS,
    COUNT(DISTINCT p.PROJECT_ID) AS PROJECT_COUNT,
    AVG(p.PROFIT_MARGIN) AS AVG_PROJECT_MARGIN,
    CASE WHEN cl.STATUS = 'CHURNED' THEN 1 ELSE 0 END AS CHURNED_LABEL
FROM INNOCEAN_INTELLIGENCE.RAW.CLIENTS cl
LEFT JOIN INNOCEAN_INTELLIGENCE.RAW.CAMPAIGNS c ON cl.CLIENT_ID = c.CLIENT_ID
LEFT JOIN INNOCEAN_INTELLIGENCE.RAW.PROJECTS p ON cl.CLIENT_ID = p.CLIENT_ID
WHERE cl.STATUS IN ('ACTIVE', 'CHURNED')
GROUP BY 
    cl.CLIENT_ID, cl.CLIENT_SEGMENT, cl.RELATIONSHIP_TYPE, cl.ANNUAL_BUDGET,
    cl.SATISFACTION_SCORE, cl.NPS_SCORE, cl.CONTRACT_START_DATE, cl.CONTRACT_END_DATE, cl.STATUS

In [ ]:
# Prepare features for churn prediction
df_client = client_data.copy()
print(f"Client records: {len(df_client)}")

# Encode categorical variables
le_segment_c = LabelEncoder()
le_relationship_c = LabelEncoder()

df_client['CLIENT_SEGMENT_ENC'] = le_segment_c.fit_transform(df_client['CLIENT_SEGMENT'].fillna('UNKNOWN'))
df_client['RELATIONSHIP_TYPE_ENC'] = le_relationship_c.fit_transform(df_client['RELATIONSHIP_TYPE'].fillna('UNKNOWN'))

# Select features
churn_features = [
    'ANNUAL_BUDGET', 'SATISFACTION_SCORE', 'NPS_SCORE', 'TENURE_DAYS',
    'DAYS_TO_CONTRACT_END', 'CAMPAIGN_COUNT', 'TOTAL_CAMPAIGN_SPEND',
    'AVG_CAMPAIGN_ROAS', 'PROJECT_COUNT', 'AVG_PROJECT_MARGIN',
    'CLIENT_SEGMENT_ENC', 'RELATIONSHIP_TYPE_ENC'
]

X_churn = df_client[churn_features].fillna(0)
y_churn = df_client['CHURNED_LABEL']

# Train-test split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_churn, y_churn, test_size=0.2, random_state=42)

# Train Gradient Boosting model
churn_model = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
churn_model.fit(X_train_c, y_train_c)

# Evaluate
y_pred_c = churn_model.predict(X_test_c)
y_prob_c = churn_model.predict_proba(X_test_c)[:, 1]

print(f"\nClient Churn Model Results:")
print(f"Accuracy: {accuracy_score(y_test_c, y_pred_c):.3f}")
print(f"ROC AUC: {roc_auc_score(y_test_c, y_prob_c):.3f}")
print(f"\nFeature Importance:")
for feat, imp in sorted(zip(churn_features, churn_model.feature_importances_), key=lambda x: -x[1])[:5]:
    print(f"  {feat}: {imp:.3f}")

In [ ]:
%%sql -r media_data
SELECT 
    mp.CHANNEL,
    mp.TARGETING_TYPE,
    c.CAMPAIGN_OBJECTIVE,
    SUM(mp.ACTUAL_SPEND) AS CHANNEL_SPEND,
    SUM(perf.IMPRESSIONS) AS TOTAL_IMPRESSIONS,
    SUM(perf.CLICKS) AS TOTAL_CLICKS,
    SUM(perf.CONVERSIONS) AS TOTAL_CONVERSIONS,
    SUM(perf.CONVERSION_VALUE) AS TOTAL_CONVERSION_VALUE,
    CASE WHEN SUM(mp.ACTUAL_SPEND) > 0 
         THEN SUM(perf.CONVERSION_VALUE) / SUM(mp.ACTUAL_SPEND) 
         ELSE 0 END AS CHANNEL_ROAS,
    CASE WHEN SUM(perf.IMPRESSIONS) > 0 
         THEN SUM(perf.CLICKS) / SUM(perf.IMPRESSIONS) 
         ELSE 0 END AS CHANNEL_CTR
FROM INNOCEAN_INTELLIGENCE.RAW.MEDIA_PLACEMENTS mp
JOIN INNOCEAN_INTELLIGENCE.RAW.MEDIA_PERFORMANCE perf ON mp.PLACEMENT_ID = perf.PLACEMENT_ID
JOIN INNOCEAN_INTELLIGENCE.RAW.CAMPAIGNS c ON mp.CAMPAIGN_ID = c.CAMPAIGN_ID
WHERE mp.STATUS = 'COMPLETED'
GROUP BY mp.CHANNEL, mp.TARGETING_TYPE, c.CAMPAIGN_OBJECTIVE

In [ ]:
# Prepare features for budget optimization
df_media = media_data.copy()
print(f"Media channel records: {len(df_media)}")

# Encode categorical variables
le_channel = LabelEncoder()
le_targeting = LabelEncoder()
le_objective_m = LabelEncoder()

df_media['CHANNEL_ENC'] = le_channel.fit_transform(df_media['CHANNEL'].fillna('UNKNOWN'))
df_media['TARGETING_TYPE_ENC'] = le_targeting.fit_transform(df_media['TARGETING_TYPE'].fillna('UNKNOWN'))
df_media['CAMPAIGN_OBJECTIVE_ENC'] = le_objective_m.fit_transform(df_media['CAMPAIGN_OBJECTIVE'].fillna('UNKNOWN'))

# Select features for ROAS prediction
budget_features = [
    'CHANNEL_SPEND', 'TOTAL_IMPRESSIONS', 'TOTAL_CLICKS', 'CHANNEL_CTR',
    'CHANNEL_ENC', 'TARGETING_TYPE_ENC', 'CAMPAIGN_OBJECTIVE_ENC'
]

X_budget = df_media[budget_features].fillna(0)
y_budget = df_media['CHANNEL_ROAS'].fillna(0)

# Standardize features
scaler = StandardScaler()
X_budget_scaled = scaler.fit_transform(X_budget)

# Train-test split
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_budget_scaled, y_budget, test_size=0.2, random_state=42)

# Train Gradient Boosting Regressor for ROAS prediction
from sklearn.ensemble import GradientBoostingRegressor
budget_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
budget_model.fit(X_train_b, y_train_b)

# Evaluate
y_pred_b = budget_model.predict(X_test_b)
from sklearn.metrics import mean_squared_error, r2_score

print(f"\nBudget Optimization Model Results:")
print(f"R² Score: {r2_score(y_test_b, y_pred_b):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_b, y_pred_b)):.3f}")
print(f"\nFeature Importance:")
for feat, imp in sorted(zip(budget_features, budget_model.feature_importances_), key=lambda x: -x[1])[:5]:
    print(f"  {feat}: {imp:.3f}")

In [ ]:
# Create visualization of model results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Campaign Performance Feature Importance
ax1 = axes[0]
top_features_camp = sorted(zip(features, campaign_model.feature_importances_), key=lambda x: -x[1])[:6]
ax1.barh([f[0][:15] for f in top_features_camp], [f[1] for f in top_features_camp], color='steelblue')
ax1.set_xlabel('Importance')
ax1.set_title('Campaign Performance\nTop Features')

# Client Churn Feature Importance
ax2 = axes[1]
top_features_churn = sorted(zip(churn_features, churn_model.feature_importances_), key=lambda x: -x[1])[:6]
ax2.barh([f[0][:15] for f in top_features_churn], [f[1] for f in top_features_churn], color='coral')
ax2.set_xlabel('Importance')
ax2.set_title('Client Churn\nTop Features')

# Budget Optimization Feature Importance
ax3 = axes[2]
top_features_budget = sorted(zip(budget_features, budget_model.feature_importances_), key=lambda x: -x[1])[:6]
ax3.barh([f[0][:15] for f in top_features_budget], [f[1] for f in top_features_budget], color='mediumseagreen')
ax3.set_xlabel('Importance')
ax3.set_title('Budget Optimization\nTop Features')

plt.tight_layout()
plt.show()

In [ ]:
# Initialize Snowflake Model Registry
registry = Registry(session=session, database_name="INNOCEAN_INTELLIGENCE", schema_name="ANALYTICS")

# Create sample input for schema inference
sample_campaign_input = pd.DataFrame({
    'TOTAL_BUDGET': [500000.0],
    'MEDIA_BUDGET': [400000.0],
    'BUDGET_RATIO': [0.8],
    'PRODUCTION_RATIO': [0.15],
    'TARGET_IMPRESSIONS': [10000000.0],
    'TARGET_ROAS': [4.0],
    'SATISFACTION_SCORE': [8.0],
    'NPS_SCORE': [50.0],
    'CLIENT_ANNUAL_BUDGET': [5000000.0],
    'CAMPAIGN_TYPE_ENC': [1],
    'CAMPAIGN_OBJECTIVE_ENC': [2],
    'CLIENT_SEGMENT_ENC': [0],
    'RELATIONSHIP_TYPE_ENC': [1]
})

print("Registering Campaign Performance Model...")
try:
    registry.log_model(
        model=campaign_model,
        model_name="CAMPAIGN_PERFORMANCE_PREDICTOR",
        version_name="v1",
        sample_input_data=sample_campaign_input,
        comment="Predicts campaign success likelihood based on budget, targeting, and client factors"
    )
    print("Campaign Performance Model registered successfully!")
except Exception as e:
    print(f"Note: {e}")

In [ ]:
# Sample input for churn model
sample_churn_input = pd.DataFrame({
    'ANNUAL_BUDGET': [1000000.0],
    'SATISFACTION_SCORE': [7.5],
    'NPS_SCORE': [30.0],
    'TENURE_DAYS': [365.0],
    'DAYS_TO_CONTRACT_END': [180.0],
    'CAMPAIGN_COUNT': [5.0],
    'TOTAL_CAMPAIGN_SPEND': [500000.0],
    'AVG_CAMPAIGN_ROAS': [3.5],
    'PROJECT_COUNT': [10.0],
    'AVG_PROJECT_MARGIN': [0.15],
    'CLIENT_SEGMENT_ENC': [1],
    'RELATIONSHIP_TYPE_ENC': [0]
})

print("Registering Client Churn Model...")
try:
    registry.log_model(
        model=churn_model,
        model_name="CLIENT_CHURN_PREDICTOR",
        version_name="v1",
        sample_input_data=sample_churn_input,
        comment="Predicts client churn risk based on engagement, satisfaction, and financial metrics"
    )
    print("Client Churn Model registered successfully!")
except Exception as e:
    print(f"Note: {e}")

In [ ]:
# Sample input for budget model
sample_budget_input = pd.DataFrame({
    'CHANNEL_SPEND': [100000.0],
    'TOTAL_IMPRESSIONS': [5000000.0],
    'TOTAL_CLICKS': [50000.0],
    'CHANNEL_CTR': [0.01],
    'CHANNEL_ENC': [2],
    'TARGETING_TYPE_ENC': [1],
    'CAMPAIGN_OBJECTIVE_ENC': [0]
})

print("Registering Budget Optimization Model...")
try:
    registry.log_model(
        model=budget_model,
        model_name="BUDGET_OPTIMIZATION_MODEL",
        version_name="v1",
        sample_input_data=sample_budget_input,
        comment="Predicts channel ROAS to optimize media budget allocation"
    )
    print("Budget Optimization Model registered successfully!")
except Exception as e:
    print(f"Note: {e}")

In [ ]:
%%sql -r models_list
SHOW MODELS IN SCHEMA INNOCEAN_INTELLIGENCE.ANALYTICS

In [ ]:
print("="*60)
print("INNOCEAN INTELLIGENCE ML MODELS - TRAINING COMPLETE")
print("="*60)
print("\nModels Trained and Registered:")
print("\n1. CAMPAIGN_PERFORMANCE_PREDICTOR")
print("   - Type: Random Forest Classifier")
print("   - Purpose: Predict campaign success likelihood")
print(f"   - Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"   - ROC AUC: {roc_auc_score(y_test, y_prob):.3f}")

print("\n2. CLIENT_CHURN_PREDICTOR")
print("   - Type: Gradient Boosting Classifier")
print("   - Purpose: Identify at-risk clients")
print(f"   - Accuracy: {accuracy_score(y_test_c, y_pred_c):.3f}")
print(f"   - ROC AUC: {roc_auc_score(y_test_c, y_prob_c):.3f}")

print("\n3. BUDGET_OPTIMIZATION_MODEL")
print("   - Type: Gradient Boosting Regressor")
print("   - Purpose: Predict channel ROAS for optimization")
print(f"   - R² Score: {r2_score(y_test_b, y_pred_b):.3f}")

print("\n" + "="*60)
print("NEXT STEPS:")
print("="*60)
print("1. Run sql/ml/07_create_model_wrapper_functions.sql")
print("2. Add model procedures as tools in Intelligence Agent")
print("3. Test predictions with sample queries")
print("\nSee docs/NOTEBOOK_ML_GUIDE.md for detailed instructions.")